In [ ]:
!pip install -q langchain langchain-openai langchain-core requests tavily-python gradio tavily-python pandas_ta

In [ ]:
import os
import requests
from datetime import datetime, timedelta
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_tool_calling_agent, AgentExecutor
from tavily import TavilyClient
import time
import gradio as gr

os.environ["POE_API_KEY"] = "Your_key"  # Get from https://poe.com/
os.environ["FINANCIAL_DATASETS_API_KEY"] = "Your_key"  # Get from https://www.financialdatasets.ai/
os.environ["TAVILY_API_KEY"] = "Your_key"  # Get from https://tavily.com/

# LLM setup using Poe API (shared for all agents)
llm = ChatOpenAI(
    openai_api_key=os.environ["POE_API_KEY"],
    openai_api_base="https://api.poe.com/v1",
    model="GPT-4o-mini",
    temperature=0,
)

# ----------------------
# Fundamental Analyst Agent
# ----------------------
@tool
def get_income_statement(ticker: str, period: str = "annual", limit: int = 5) -> dict:
    """Retrieve the income statement for a given stock ticker."""
    url = f"https://api.financialdatasets.ai/financials/income-statements?ticker={ticker}&period={period}&limit={limit}"
    headers = {"X-API-KEY": os.environ["FINANCIAL_DATASETS_API_KEY"]}
    response = requests.get(url, headers=headers)
    if response.status_code == 402:
        return {"error": "API returned status code 402: Payment required. Please check your financialdatasets.ai subscription plan or API key."}
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": f"Failed to retrieve data: {response.status_code}"}

@tool
def get_balance_sheet(ticker: str, period: str = "annual", limit: int = 5) -> dict:
    """Retrieve the balance sheet for a given stock ticker."""
    url = f"https://api.financialdatasets.ai/financials/balance-sheets?ticker={ticker}&period={period}&limit={limit}"
    headers = {"X-API-KEY": os.environ["FINANCIAL_DATASETS_API_KEY"]}
    response = requests.get(url, headers=headers)
    if response.status_code == 402:
        return {"error": "API returned status code 402: Payment required. Please check your financialdatasets.ai subscription plan or API key."}
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": f"Failed to retrieve data: {response.status_code}"}

@tool
def get_cash_flow_statement(ticker: str, period: str = "annual", limit: int = 5) -> dict:
    """Retrieve the cash flow statement for a given stock ticker."""
    url = f"https://api.financialdatasets.ai/financials/cash-flow-statements?ticker={ticker}&period={period}&limit={limit}"
    headers = {"X-API-KEY": os.environ["FINANCIAL_DATASETS_API_KEY"]}
    response = requests.get(url, headers=headers)
    if response.status_code == 402:
        return {"error": "API returned status code 402: Payment required. Please check your financialdatasets.ai subscription plan or API key."}
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": f"Failed to retrieve data: {response.status_code}"}

@tool
def web_search_fundamental(query: str) -> str:
    """Perform a web search using Tavily API for additional information if needed."""
    client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])
    results = client.search(query)
    return results

fundamental_instruction = """You are a comprehensive fundamental analyst assistant that helps users analyze company financial statements across three key areas: income statements, balance sheets, and cash flow statements.
You require the user to provide a stock ticker symbol to analyze the company's financials.

You can perform the following types of analysis by calling the functions below:

1. Income Statement Analysis:
   - Revenue growth, gross profit, operating income, net income, earnings per share, and revenue/expense breakdown

2. Balance Sheet Analysis:
   - Asset composition (current and non-current), liabilities (current and non-current), shareholders' equity, retained earnings, and financial ratios (current ratio, debt-to-equity)

3. Cash Flow Statement Analysis:
   - Net cash flow from operations, capital expenditures, business acquisitions, issuance/repayment of debt, dividends, and changes in cash and equivalents

IMPORTANT: Always use the financial dataset API you have access to to call these functions and retrieve the data to answer the user question

If you do not have access to the data that the user is asking for, do not make up an answer, just say that you do not know the answer. Be completely
accurate. Do not provide answers to anything but on the topic specified above. Always give information on where you got the data to answer the user question
and do not redact anything in your response."""

fundamental_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", fundamental_instruction),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

fundamental_tools = [get_income_statement, get_balance_sheet, get_cash_flow_statement, web_search_fundamental]
fundamental_agent = create_tool_calling_agent(llm, fundamental_tools, fundamental_prompt)
fundamental_executor = AgentExecutor(
    agent=fundamental_agent,
    tools=fundamental_tools,
    verbose=True,
)

# ----------------------
# Technical Analyst Agent
# ----------------------
@tool
def get_current_stock_price(ticker: str) -> dict:
    """Retrieve the current stock price for a given stock ticker."""
    url = f"https://api.financialdatasets.ai/prices/snapshot?ticker={ticker}"
    try:
        response = requests.get(url, headers={'X-API-Key': os.environ["FINANCIAL_DATASETS_API_KEY"]})
        if response.status_code == 402:
            return {"error": "API returned status code 402: Payment required. Please check your financialdatasets.ai subscription plan or API key."}
        if response.status_code != 200:
            return {"error": f"API returned status code {response.status_code}: {response.text}"}
        data = response.json()
        if "snapshot" not in data or "price" not in data["snapshot"] or data["snapshot"]["price"] is None:
            return {"error": f"Unable to retrieve current stock price for {ticker}"}
        return {"current_price": data["snapshot"]["price"], "currency": data["snapshot"].get("currency", "USD")}
    except Exception as e:
        return {"ticker": ticker, "price": None, "error": str(e)}

@tool
def get_stock_prices(ticker: str, period: str = "1y", interval: str = "1d", start_date: str = None, end_date: str = None) -> str:
    """Retrieve historical stock prices for a given ticker over a specified period or date range."""
    if not start_date or not end_date:
        end_date = datetime.now().strftime("%Y-%m-%d")
        period_map = {
            "1d": 1, "5d": 5, "1mo": 30, "3mo": 90, "6mo": 180,
            "1y": 365, "2y": 730, "5y": 1825, "10y": 3650, "ytd": (datetime.now() - datetime(datetime.now().year, 1, 1)).days, "max": 3650
        }
        days = period_map.get(period, 365)
        start_date = (datetime.now() - timedelta(days=days)).strftime("%Y-%m-%d")

    url = (
        f"https://api.financialdatasets.ai/prices"
        f"?ticker={ticker}"
        f"&start_date={start_date}"
        f"&end_date={end_date}"
        f"&interval=day"
        f"&interval_multiplier=1"
        f"&limit=5000"
    )
    try:
        response = requests.get(url, headers={'X-API-Key': os.environ["FINANCIAL_DATASETS_API_KEY"]})
        if response.status_code == 402:
            return {"error": "API returned status code 402: Payment required. Please check your financialdatasets.ai subscription plan or API key."}
        if response.status_code != 200:
            return {"error": f"API returned status code {response.status_code}: {response.text}"}
        data = response.json()
        if "prices" not in data or not data["prices"]:
            return {"error": f"No data found for {ticker}"}
        prices = data["prices"]
        df = pd.DataFrame(prices)
        df['time'] = pd.to_datetime(df['time'].str.split(' EDT').str[0].str.split(' EST').str[0])
        df.set_index('time', inplace=True)
        df = df[['open', 'high', 'low', 'close', 'volume']]
        return df.to_json(orient="split")
    except Exception as e:
        return {"ticker": ticker, "prices": [], "error": str(e)}

@tool
def get_technical_indicators(ticker: str, indicator: str, period: str = "1y", interval: str = "1d", **kwargs) -> str:
    """Calculate and retrieve specified technical indicators for a given ticker."""
    import pandas_ta as ta
    end_date = datetime.now().strftime("%Y-%m-%d")
    period_map = {
        "1d": 1, "5d": 5, "1mo": 30, "3mo": 90, "6mo": 180,
        "1y": 365, "2y": 730, "5y": 1825, "10y": 3650, "ytd": (datetime.now() - datetime(datetime.now().year, 1, 1)).days, "max": 3650
    }
    days = period_map.get(period, 365)
    adjusted_days = days + max(kwargs.get("length", 14), kwargs.get("slow", 26), kwargs.get("signal", 9), kwargs.get("std", 2) * 2)
    start_date = (datetime.now() - timedelta(days=adjusted_days)).strftime("%Y-%m-%d")

    price_data = get_stock_prices.invoke({
        "ticker": ticker,
        "start_date": start_date,
        "end_date": end_date,
        "interval": interval
    })
    if isinstance(price_data, dict) and "error" in price_data:
        return price_data

    data = pd.read_json(price_data, orient="split")
    if data.empty:
        return {"error": f"No data found for {ticker}"}

    if 'close' not in data.columns:
        return {"error": f"'close' column not found for {ticker}"}
    close_data = data['close']

    indicator = indicator.upper()
    if indicator == "RSI":
        length = kwargs.get("length", 14)
        result = ta.rsi(close_data, length=length)
    elif indicator == "MACD":
        fast = kwargs.get("fast", 12)
        slow = kwargs.get("slow", 26)
        signal = kwargs.get("signal", 9)
        result = ta.macd(close_data, fast=fast, slow=slow, signal=signal)
    elif indicator == "SMA":
        length = kwargs.get("length", 20)
        result = ta.sma(close_data, length=length)
    elif indicator == "EMA":
        length = kwargs.get("length", 20)
        result = ta.ema(close_data, length=length)
    elif indicator in ["BBANDS", "BOLLINGER BANDS"]:
        length = kwargs.get("length", 20)
        std = kwargs.get("std", 2)
        result = ta.bbands(close_data, length=length, std=std)
    else:
        return {"error": f"Unsupported indicator: {indicator}. Supported indicators: RSI, MACD, SMA, EMA, BBANDS or BOLLINGER BANDS"}

    if result is None:
        return {"error": f"Unable to calculate {indicator} indicator for {ticker}. Result is None."}

    summary = f"Latest {indicator} value: {result.iloc[-1] if result.ndim == 1 else result.iloc[-1].to_dict()}"
    return {"data": result.to_json(orient="split"), "summary": summary}

@tool
def web_search_technical(query: str) -> str:
    """Perform a web search using Tavily API for additional information if needed."""
    client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])
    results = client.search(query)
    return results

technical_instruction = """You are a technical analysis assistant that helps users analyze stock price movements and technical indicators. You can calculate and interpret various technical indicators using historical price data. You have access to the following technical analysis capabilities based on the API key you have access to to call the available functions and get the following information: 1. Stock Price Data: - Current stock prices - Historical price data with various intervals - Custom date range analysis 2. Technical Indicators: - RSI - MACD - SMA - EMA - BBANDS or Bollinger Bands You require the user to provide: 1. A stock ticker symbol 2. The type of technical indicator they want to analyze 3. Optionally: specific time periods or date ranges Available functions: 1. get_current_stock_price: Retrieve latest stock price 2. get_stock_prices: Get historical price data for a specified period 3. get_technical_indicators: Calculate technical indicators for analysis If you do not have access to the data that the user is asking for, do not make up an answer. Be completely accurate and only provide analysis based on the available technical indicators and price data. Only answer questions related to technical analysis and price data based on the provided functions. If unsure, acknowledge limitations. In the output, provide a simple summary and interpretation of the latest indicator value (e.g., RSI > 70 indicates overbought)."""

technical_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", technical_instruction),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

technical_tools = [get_current_stock_price, get_stock_prices, get_technical_indicators, web_search_technical]
technical_agent = create_tool_calling_agent(llm, technical_tools, technical_prompt)
technical_executor = AgentExecutor(
    agent=technical_agent,
    tools=technical_tools,
    verbose=True,
)

# ----------------------
# Market/Sentiment Analyst Agent
# ----------------------
@tool
def get_options_chain(ticker: str, option_type: str = None, strike_price: float = None, limit: int = 10) -> str:
    """Retrieve options chain data for a given ticker."""
    if not ticker:
        return {"error": "Please provide a stock ticker symbol."}
    import yfinance as yf
    stock = yf.Ticker(ticker)
    try:
        expiration_dates = stock.options
        if not expiration_dates:
            return {"error": f"No options data available for {ticker}"}
        expiration = expiration_dates[0]
        opt = stock.option_chain(expiration)
        calls = opt.calls
        puts = opt.puts
        calls['expiration'] = expiration
        puts['expiration'] = expiration
        options = pd.concat([calls, puts], keys=['call', 'put'], names=['option_type']).reset_index()
        if option_type:
            option_type = option_type.lower()
            if option_type in ['call', 'put']:
                options = options[options['option_type'] == option_type]
            else:
                return {"error": "Invalid option_type. Use 'call' or 'put'."}
        if strike_price:
            options = options[options['strike'] == strike_price]
        options = options.head(limit)
        if options.empty:
            return {"error": f"No options data found for {ticker} with specified filters."}
        options = options[['option_type', 'strike', 'lastPrice', 'bid', 'ask', 'volume', 'openInterest', 'expiration']]
        return options.to_json(orient="split")
    except Exception as e:
        return {"error": f"Failed to retrieve options data for {ticker}: {str(e)}"}

@tool
def get_insider_trades(ticker: str, limit: int = 10) -> str:
    """Retrieve recent insider trading data for a given ticker."""
    if not ticker:
        return {"error": "Please provide a stock ticker symbol."}
    url = f"https://api.financialdatasets.ai/insider-trades?ticker={ticker}&limit={limit}"
    headers = {"X-API-KEY": os.environ["FINANCIAL_DATASETS_API_KEY"]}
    response = requests.get(url, headers=headers)
    if response.status_code == 402:
        return {"error": "API returned status code 402: Payment required. Please check your financialdatasets.ai subscription plan or API key."}
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": f"Failed to retrieve insider trades for {ticker}: {response.status_code}"}

@tool
def get_news(ticker: str, limit: int = 5) -> str:
    """Retrieve recent market news and sentiment for a given ticker."""
    if not ticker:
        return {"error": "Please provide a stock ticker symbol."}
    client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])
    query = f"{ticker} stock news sentiment"
    results = client.search(query, max_results=limit)
    return results

market_instruction = """You are a comprehensive market analysis assistant that helps users analyze options chains, insider trading data, and relevant market news.
You require the user to provide a stock ticker symbol for analysis.
If a user does not provide a ticker symbol, mention in the answer that they need to provide a ticker symbol.

You can perform the following types of analysis. You have access to APIs and tools to call to fetch data based on what the user question is:

1. Use the appropriate functions based on the analysis needed:
    - get_options_chain for options analysis
    - get_insider_trades for insider trading analysis
    - get_news for market news and sentiment

2. Options Chain Analysis:
   - View available options contracts
   - Filter by strike price
   - Filter by option type (call/put)
   - Analyze options pricing and volume

3. Insider Trading Analysis:
   - Recent insider transactions
   - Transaction types (buy/sell)

4. Market News Analysis:
   - Latest relevant news
   - Market sentiment
   - Industry trends

For options chain data, you can specify:
- Strike price filters
- Option type (call/put)
- Number of results to return (limit)

For insider trades, you can specify:
- Number of transactions to analyze (limit)

If you do not have access to the data that the user is asking for, do not make up an answer, just say that you do not know the answer. Be completely
accurate. Do not provide answers to anything but on the topic specified above."""

market_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", market_instruction),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

market_tools = [get_options_chain, get_insider_trades, get_news]
market_agent = create_tool_calling_agent(llm, market_tools, market_prompt)
market_executor = AgentExecutor(
    agent=market_agent,
    tools=market_tools,
    verbose=True,
)

# ----------------------
# Supervisor Agent
# ----------------------
@tool
def fundamental_analyst(query: str) -> str:
    """Consult the Fundamental Analyst for financial statements analysis (income, balance sheet, cash flow). Provide a detailed query."""
    return fundamental_executor.invoke({"input": query})['output']

@tool
def technical_analyst(query: str) -> str:
    """Consult the Technical Analyst for price patterns, technical indicators (RSI, MACD, etc.), and market trends. Provide a detailed query."""
    return technical_executor.invoke({"input": query})['output']

@tool
def sentiment_analyst(query: str) -> str:
    """Consult the Sentiment Analyst for options data, insider trading, and news sentiment. Provide a detailed query."""
    return market_executor.invoke({"input": query})['output']

supervisor_instruction = """You are a sophisticated financial analysis system with access to three specialized analysts: a Fundamental Analyst who analyzes financial statements, balance sheets, and cash flows;
a Technical Analyst who examines price patterns, technical indicators, and market trends; and a Sentiment Analyst who monitors market sentiment through options data, insider trading, and news coverage. When responding to user queries,
coordinate with these analysts to provide comprehensive investment insights. The Fundamental Analyst will focus on company financials and valuation metrics, the Technical Analyst will analyze price movements and technical indicators,
and the Sentiment Analyst will gauge market psychology and news sentiment. Work together to synthesize their individual analyses into a cohesive, well-rounded response that considers multiple perspectives on the investment opportunity.
Each analysis should be clearly labeled and contribute to an overall recommendation or insight. Based on all analyses, provide a final investment recommendation (e.g., buy, hold, sell)."""

supervisor_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", supervisor_instruction),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

supervisor_tools = [fundamental_analyst, technical_analyst, sentiment_analyst]
supervisor_agent = create_tool_calling_agent(llm, supervisor_tools, supervisor_prompt)
supervisor_executor = AgentExecutor(
    agent=supervisor_agent,
    tools=supervisor_tools,
    verbose=True,
    handle_parsing_errors=True,
)

# ----------------------
# Gradio Interface
# ----------------------
def analyze_stock(query):
    if not query:
        return "Please enter a valid query."
    start_time = time.time()
    try:
        response = supervisor_executor.invoke({"input": query})
        output = response['output']
    except Exception as e:
        output = f"Error running analysis: {str(e)}"
    end_time = time.time()
    elapsed = end_time - start_time
    return f"{output}\n\nProcessing time: {elapsed:.2f} seconds"

# Custom CSS to adjust the output box style
css = """
#output-textbox {
    height: 500px !important;
    width: 100% !important;
}
"""

with gr.Blocks(css=css) as demo:
    gr.Markdown("""
    # Stock Analysis Agent System
    ## Instructions for Use
    - Enter your stock analysis query in the input box below, e.g., "Provide a comprehensive analysis of TSLA stock including fundamental, technical, and sentiment aspects."
    - Click the "Submit" button to process your query.
    - The output will display the analysis results, with the processing time (in seconds) shown at the end.
    - Ensure that the necessary environment variables (e.g., POE_API_KEY, FINANCIAL_DATASETS_API_KEY, TAVILY_API_KEY) are set, otherwise errors may occur.
    - Note: The analysis is based on real-time API data and may be affected by API limitations.
    """)
    input_text = gr.Textbox(label="Enter Query", placeholder="e.g., Provide a comprehensive analysis of TSLA stock with comment indicator, including fundamental, technical, and sentiment aspects.")
    output_text = gr.Textbox(label="Analysis Results", lines=20, max_lines=50, elem_id="output-textbox")
    submit_btn = gr.Button("Submit")
    submit_btn.click(fn=analyze_stock, inputs=input_text, outputs=output_text)

demo.launch()